# Tutorial 6: The Expressiveness Tradeoff

*What Types Prevent*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-typed-passages/blob/main/notebooks/06-the-expressiveness-tradeoff.ipynb)

---

## A Note from the Archives

> *Year 815, Capital Archives*
>
> "You've crippled the calculus," Torren Gast accused.
>
> Brennis faced his critic calmly. "I've made it safe."
>
> "Safe? The identity function — the simplest passage imaginable — can no longer be applied to all arguments! In your typed system, we need a different identity for naturals, a different identity for booleans, a different identity for functions..."
>
> "Yes," Brennis agreed. "That is the price of termination."
>
> "And the Y combinator? Recursion itself is forbidden?"
>
> "The Y combinator creates non-termination. I've proposed alternatives."
>
> The debate would define the next century of passage theory.

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Explain why the identity function must be monomorphic
2. Explain why the Y combinator is untypable
3. Describe the safety/expressiveness tradeoff
4. Understand primitive recursion as a typed alternative

## Setup

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

expressions_df = pd.read_csv(BASE_URL + "typed_passage_expressions.csv")
comparison_df = pd.read_csv(BASE_URL + "type_system_comparison.csv")

print(f"Loaded {len(comparison_df)} comparison cases")

## 1. The Identity Function Problem

In untyped lambda calculus, the identity function works for everything:

$$\text{ID} = \lambda x.x$$

- $\text{ID}\ 5 = 5$
- $\text{ID}\ \text{TRUE} = \text{TRUE}$
- $\text{ID}\ (\lambda y.y) = \lambda y.y$

But in simply typed lambda calculus, we must choose a type:

- $\lambda x:\text{nat}.x : \text{nat} \to \text{nat}$
- $\lambda x:\text{bool}.x : \text{bool} \to \text{bool}$

These are **different** functions with **different** types!

In [ ]:
# Identity function comparison
identity_cases = comparison_df[comparison_df['expression_name'].str.contains('dentity', na=False)]

print("The Identity Function:")
print()
for _, row in identity_cases.iterrows():
    typable = "YES" if row['typable'] == True else ("PARTIAL" if row['typable'] == 'PARTIAL' else "NO")
    print(f"  {row['expression_name']}:")
    print(f"    Untyped: {row['untyped_form']}")
    print(f"    Typed: {row['typed_form']}")
    print(f"    Typable: {typable}")
    print(f"    Note: {row['expressiveness_note']}")
    print()

## 2. Why Polymorphism is Lost

The identity function is **polymorphic** — it works for any type. But in STLC:

- Every expression has exactly **one** type
- $\lambda x.x$ must be assigned a specific $\tau \to \tau$
- We can't say "for all types $\tau$, this works"

This is called **monomorphism**. Later type systems (System F) will recover polymorphism.

In [ ]:
# Monomorphism examples
print("Monomorphism in STLC:")
print()
print("  In untyped calculus:")
print("    ID = λx.x")
print("    ID 5 = 5")
print("    ID TRUE = TRUE")
print("    Same function, different uses")
print()
print("  In simply typed calculus:")
print("    ID_nat = λx:nat.x : nat→nat")
print("    ID_bool = λx:bool.x : bool→bool")
print("    ID_nat 5 = 5      ✓")
print("    ID_nat TRUE = ?   ✗ Type error!")
print("    Different functions for different types")

## 3. The Y Combinator is Untypable

The Y combinator enables general recursion:

$$Y = \lambda f.(\lambda x.f(x\ x))(\lambda x.f(x\ x))$$

But it contains $(x\ x)$ — self-application. For this to be typed:

- $x$ must have type $\tau$
- $x$ is applied to $x$, so $\tau = \tau \to \sigma$
- No finite type satisfies this!

**Y is untypable**, so general recursion is unavailable in STLC.

In [ ]:
# Y combinator analysis
y_case = comparison_df[comparison_df['expression_name'] == 'Y combinator']

print("The Y Combinator:")
print()
if not y_case.empty:
    row = y_case.iloc[0]
    print(f"  Expression: {row['untyped_form']}")
    print(f"  Typable: {row['typable']}")
    print(f"  Terminates: {row['terminates_untyped']}")
    print(f"  Note: {row['expressiveness_note']}")
print()
print("  Without Y, we cannot define:")
print("    - Factorial")
print("    - Fibonacci")
print("    - Any general recursive function")

## 4. Factorial: With Y vs Without

**With Y combinator** (untyped, but may not terminate):

$$\text{FACT} = Y\ (\lambda f.\lambda n. \text{if } (\text{ISZERO } n) \text{ then } 1 \text{ else } (\text{MULT } n (f (\text{PRED } n))))$$

**With primitive recursion** (typed, always terminates):

$$\text{FACT} = \text{REC } 1 (\lambda n.\lambda r. \text{MULT } (\text{SUCC } n) r)$$

In [ ]:
# Factorial comparison
factorial_cases = comparison_df[comparison_df['expression_name'].str.contains('actorial', na=False)]

print("Factorial: Two Approaches")
print()
for _, row in factorial_cases.iterrows():
    typable = "YES" if row['typable'] == True else "NO"
    term_typed = row['terminates_typed'] if pd.notna(row['terminates_typed']) else "N/A"
    print(f"  {row['expression_name']}:")
    print(f"    Typable: {typable}")
    print(f"    Terminates (typed): {term_typed}")
    print(f"    Note: {row['expressiveness_note']}")
    print()

## 5. Primitive Recursion: The Typed Alternative

Brennis Mund proposed **primitive recursion** as a safe alternative:

$$\text{REC} : \tau \to (\text{nat} \to \tau \to \tau) \to \text{nat} \to \tau$$

- First argument: base case (value at 0)
- Second argument: step function (how to go from $n$ to $n+1$)
- Third argument: the natural number to recurse on

This is typable AND always terminates!

In [ ]:
# Primitive recursion examples
print("Primitive Recursion:")
print()
print("  REC : τ → (nat → τ → τ) → nat → τ")
print()
print("  REC base step 0     = base")
print("  REC base step (n+1) = step n (REC base step n)")
print()
print("  Example: Factorial")
print("    FACT = REC 1 (λn.λr. MULT (SUCC n) r)")
print("    FACT 0 = 1")
print("    FACT 1 = MULT (SUCC 0) 1 = 1")
print("    FACT 2 = MULT (SUCC 1) (FACT 1) = 2")
print("    FACT 3 = MULT (SUCC 2) (FACT 2) = 6")

## 6. The Safety/Expressiveness Spectrum

Every type system makes a tradeoff:

| Property | Untyped | Simply Typed | Dependently Typed |
|----------|---------|--------------|-------------------|
| Termination | Not guaranteed | Guaranteed | Guaranteed |
| Polymorphism | Natural | Lost | Recovered |
| Recursion | Y combinator | Primitive only | Structural |
| Expressiveness | Maximum | Limited | Moderate |
| Safety | None | Type errors caught | Proofs included |

In [ ]:
# Compare typable vs untypable expressions
print("Expressiveness Comparison:")
print()

typable_count = len(comparison_df[comparison_df['typable'] == True])
untypable_count = len(comparison_df[comparison_df['typable'] == False])
partial_count = len(comparison_df[comparison_df['typable'] == 'PARTIAL'])

print(f"  Fully typable expressions: {typable_count}")
print(f"  Partially typable: {partial_count}")
print(f"  Untypable expressions: {untypable_count}")
print()
print("  Untypable expressions typically involve:")
print("    - Self-application (x x)")
print("    - Fixed point combinators (Y, Θ)")
print("    - Omega variants")

## 7. What Types Catch

The benefit of types: many errors are caught at "compile time" (before running).

- Applying a boolean to a natural: **rejected**
- Passing the wrong number of arguments: **rejected**
- Self-application (and thus Omega): **rejected**
- Using a function where a number is expected: **rejected**

In [ ]:
# What types catch
print("Errors Caught by Types:")
print()
print("  ✗ (TRUE 5)        - Can't apply boolean to number")
print("  ✗ (5 TRUE)        - Can't apply number to boolean")
print("  ✗ (SUCC TRUE)     - SUCC expects nat, got bool")
print("  ✗ (λx.x x)        - Self-application requires infinite type")
print("  ✗ Y               - Fixed point combinator untypable")
print()
print("  All of these would 'go wrong' in untyped calculus.")
print("  Types prevent them statically.")

## 8. Beyond STLC: Future Directions

Brennis Mund knew that simply typed lambda calculus was just the beginning:

> "To those who come after — may you extend what we could only begin."
> — Dedication, "The Typed Passage Discipline"

Future scholars would develop:

- **Polymorphic types** (System F): Recover the universal identity
- **Dependent types**: Types that depend on values
- **Linear types**: Track resource usage
- **Homotopy types**: Equate equivalent structures

In [ ]:
# Preview of future type systems
print("Beyond Simply Typed Lambda Calculus:")
print()
print("  Course 4 (Arren Mott): Domain theory")
print("    - How to give meaning to non-terminating programs")
print()
print("  Course 5 (Sereth Linn): Dependent types")
print("    - Types that carry proofs")
print("    - Vector of length n: Vec τ n")
print()
print("  Course 6 (Quellen Vast): Linear types")
print("    - Track resources, prevent duplication")
print()
print("  Course 7 (Dora Keth): Homotopy type theory")
print("    - Univalence: equivalent types are equal")

## Summary

In this tutorial, we learned:

1. **Monomorphism**: The identity must have a specific type (no polymorphism)
2. **Y is untypable**: General recursion is forbidden
3. **The tradeoff**: Safety (termination, no type errors) vs Expressiveness (polymorphism, recursion)
4. **Primitive recursion**: A typed alternative that recovers some recursive patterns
5. **Types catch errors**: Many bugs are prevented statically
6. **Future work**: More expressive type systems can recover lost features

In the final tutorial, we'll synthesize everything into Brennis Mund's complete Typed Passage Discipline.

---

## Exercises

1. Write out the types of three different identity functions (for nat, bool, and nat→nat).

2. Why can't we type $\lambda f.(f\ f)$? What equation would the type of $f$ need to satisfy?

3. Express addition using primitive recursion: PLUS m n.

4. What's an advantage of catching errors at "compile time" vs "run time"?

---

*Next: Tutorial 7 - The Typed Passage Discipline*